# Objectives
---
Here, our objective is to predict how much insurance charges will be charged by a client. For insurance companies, to predict the appropriate charges is beneficial to attract healthy clients by adjusting insurance fees individually and for new clients, accurate prediction improves their customer satisfactions unless they aware their healthy risks.

# Dataset

American insurance company's data<br>
References: https://www.kaggle.com/mirichoi0218/insurance <br><br>
<br><br>

**Datasets**

- `age`: age of primary beneficiary
- `sex`: insurance contractor gender, female, male
- `bmi`: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m^2) using the ratio of height to weight, ideally 18.5 to 24.9
- `children`: Number of children covered by health insurance / Number of dependents
- `smoker`: Smoking
- `region`: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
- `charges`: Individual medical costs billed by health insurance

# Stragegy
---

 - Objectives: To predict the charges by using client information.
 - Dependent variables: `charges`
 - Independent variables: `age`, `sex`, `bmi`, `children`, `smoker`, `region`
 - Regression models: Random Forest, XGB, LightGBM

# Libraries
---

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
import pickle

from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.simplefilter('ignore')

# Check the data
---

## `.read_csv()`

In [ ]:
data = pd.read_csv("insurance.csv", engine='python', encoding='utf-8', index_col="id")

## `.shape`

In [ ]:
data.shape

## `.columns`

In [ ]:
data.columns

## `.dtypes`

In [ ]:
data.dtypes

## `.info()`

In [ ]:
data.info()

## `.value_counts()`

In [ ]:
data['sex'].value_counts()

In [ ]:
data['smoker'].value_counts()

In [ ]:
data['region'].value_counts()

## `.head()` `.tail()`

In [ ]:
data.head(10)

In [ ]:
data.tail()

## Descriptive statistics

In [ ]:
data.describe()

## Distribution and Correlation of variables

In [ ]:
g = sns.pairplot(data)

## Missing values

In [ ]:
data.isnull().sum()

# Preprocess the data
---

## Chenge object to numerical

### categorical values

In [ ]:
data["sex"] = data["sex"].replace({"male":1, "female":0})

In [ ]:
data["smoker"] = data["smoker"].replace({"no":0, "yes":1})

### multi-categorical values

In [ ]:
data_region = pd.get_dummies(data["region"], sparse=False)
df = pd.concat([data, data_region], axis=1)
df = df.drop("region", axis=1)

In [ ]:
df.head()

## Recheck the dist and corr

In [ ]:
sns.pairplot(df.drop(["northeast", "northwest", "southeast", "southwest"], axis=1))

See the outliers in `charges`. Not many customers will charge more than 40000 USD.

# Regression models

## Split the data

`train_test_split()`

In [ ]:
y_reg_target = df['charges']

In [ ]:
df.columns

In [ ]:
X_reg_explanatory = df.drop(["charges"], axis=1)

In [ ]:
X_train,X_test,y_train, y_test = train_test_split(X_reg_explanatory, y_reg_target, random_state=1, test_size=0.2)

In [ ]:
len(X_train), len(X_test)

In [ ]:
len(y_train),len(y_test)

Check the distributions of train and test data are equal.

In [ ]:
y_train.hist()

In [ ]:
y_test.hist()

Not so different. Good.

## Base models

### Random Forest
`RandomForestRegressor()`

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train,y_train)

### XGB
`xgb.XGBRegressor()`

In [ ]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train,y_train)

### LightGBM

`lgb.LGBMRegressor()`

In [ ]:
lgb_model = lgb.LGBMRegressor()
lgb_model.fit(X_train,y_train)

## Prediction
---

 `.predict()`

### RF

In [ ]:
rf_pred = rf_model.predict(X_test)
rf_pred

### XGB

In [ ]:
xgb_pred = xgb_model.predict(X_test)
xgb_pred

### LightGBM

In [ ]:
lgb_pred = lgb_model.predict(X_test)
lgb_pred

## Accuracy
---

### RF

In [ ]:
# RMSE and MAE
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_pred))
mae_rf = mean_absolute_error(y_test, rf_pred)

print("RMSE:", rmse_rf)
print("MAE:", mae_rf)

In [ ]:
# R2 score
r2_rf = r2_score(y_test, rf_pred)
r2_rf

In [ ]:
# Scatter plot
fig = plt.figure(figsize=(6,5))

plt.scatter(y_test,rf_pred,color="blue")
x = np.arange(0, 40000)
plt.plot(x,x,color="red")
plt.title("predicted_data vs test_data")
plt.xlabel("test_data")
plt.ylabel("predicted_data")
plt.show()

Some data seem to deviate from the red line.

In [ ]:
# Residual plots
plt.scatter(rf_pred, rf_pred - y_test, color = 'blue')      # Plot the residuals
plt.hlines(y = 0, xmin = -2000, xmax = 38000, color = 'black') # x-axis
plt.title('Residual Plot')                                # Title
plt.xlabel('Predicted Values')                            # x-label
plt.ylabel('Residuals')                                   # y-label
plt.grid()                                                # grid-line

plt.show()                                               # Display 

It looks messed up.

### XGB

In [ ]:
# RMSE and MAE
rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_pred))
mae_xgb = mean_absolute_error(y_test, xgb_pred)

print("RMSE:", rmse_xgb)
print("MAE:", mae_xgb)

In [ ]:
# R2 score
r2_xgb = r2_score(y_test, xgb_pred)
r2_xgb

In [ ]:
# Scatter plot
fig = plt.figure(figsize=(6,5))

plt.scatter(y_test,xgb_pred,color="blue")
x = np.arange(0, 40000)
plt.plot(x,x,color="red")
plt.title("predicted_data vs test_data")
plt.xlabel("test_data")
plt.ylabel("predicted_data")

plt.show()


In [ ]:
# residual plot
plt.scatter(xgb_pred, xgb_pred - y_test, color = 'blue')   
plt.hlines(y = 0, xmin = -2000, xmax = 38000, color = 'black') 
plt.title('Residual Plot')                                
plt.xlabel('Predicted Values')                            
plt.ylabel('Residuals')                                   
plt.grid()                                                

plt.show()                                              

### LightGBM

In [ ]:
# RMSE and MAE
rmse_lgb = np.sqrt(mean_squared_error(y_test, lgb_pred))
mae_lgb = mean_absolute_error(y_test, lgb_pred)

print("RMSE:", rmse_lgb)
print("MAE:", mae_lgb)

In [ ]:
# R2 score
r2_lgb = r2_score(y_test, lgb_pred)
r2_lgb

In [ ]:
# Scatter plot
fig = plt.figure(figsize=(6,5))

plt.scatter(y_test,lgb_pred,color="blue")
x = np.arange(0, 40000)
plt.plot(x,x,color="red")
plt.title("predicted_data vs test_data")
plt.xlabel("test_data")
plt.ylabel("predicted_data")

plt.show()


It looks better than the other above results.

In [ ]:
# Residual plot
plt.scatter(rf_pred, lgb_pred - y_test, color = 'blue')      
plt.hlines(y = 0, xmin = -2000, xmax = 38000, color = 'black') 
plt.title('Residual Plot')                                
plt.xlabel('Predicted Values')                            
plt.ylabel('Residuals')                                   
plt.grid()                                                

plt.show()                                               

The residual is often negative.

## Parameter tunings

### RF

In [ ]:
# Default
rf_model

#### Grid Search

In [ ]:
# # #Grid_Search
# params = {
#        'n_estimators'      : [50,100,150],
#        'random_state'      : [1],
#        'n_jobs'            : [-1],
#        'min_samples_split' : [5, 10, 15],
#        'max_depth'         : [3,4,5,6,7],
#        'max_leaf_nodes'    : [15, 20, 25]
# }
# grid_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=1), param_grid=params, \
#                        cv = 10, scoring = 'r2')
# grid_rf.fit(X_train, y_train)
# print(grid_rf.best_estimator_)

In [ ]:
# Best estimator
grid_rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=20,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=-1, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

#### Fitting

In [ ]:
grid_rf.fit(X_train,y_train)

#### Prediction

In [ ]:
grid_rf_pred = grid_rf.predict(X_test)

#### Accuracy

In [ ]:
# RMSE and MAE
grid_rf_rmse = np.sqrt(mean_squared_error(y_test, grid_rf_pred))
grid_rf_mae = mean_absolute_error(y_test, grid_rf_pred)

print("RMSE:", grid_rf_rmse)
print("MAE:", grid_rf_mae)

In [ ]:
# R2 score
r2_grid_rf = r2_score(y_test, grid_rf_pred)
r2_grid_rf

In [ ]:
# Scatter plot
fig = plt.figure(figsize=(6,5))

plt.scatter(y_test,grid_rf_pred,color="blue")
x = np.arange(0, 40000)
plt.plot(x,x,color="red")
plt.title("predicted_data vs test_data")
plt.xlabel("test_data")
plt.ylabel("predicted_data")

plt.show()


Improved than the base model

In [ ]:
# Residual plot
plt.scatter(grid_rf_pred, grid_rf_pred - y_test, color = 'blue')      
plt.hlines(y = 0, xmin = -2000, xmax = 38000, color = 'black') 
plt.title('Residual Plot')                                
plt.xlabel('Predicted Values')                            
plt.ylabel('Residuals')                                   
plt.grid()                                                

plt.show()                                              

The residuals still have the pattern biased to negative.

### XGB

In [ ]:
# Default
xgb_model

#### Grid Search

In [ ]:
# # #Grid_Search
# params = {
#        'n_estimators'      : [30,40,50],
#        'random_state'      : [1],
#        'n_jobs'            : [-1],
#        'min_samples_split' : [3, 5, 7],
#        'max_depth'         : [3,4,5,6,7],
#        'max_leaf_nodes'    : [3, 5, 7]
# }

# grid_xgb = GridSearchCV(estimator=xgb.XGBRegressor(random_state=1), param_grid=params, \
#                        cv = 10, scoring = 'r2')
# grid_xgb.fit(X_train, y_train)
# print(grid_xgb.best_estimator_)

In [ ]:
# Best estimator
grid_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, max_leaf_nodes=3, min_child_weight=1,
             min_samples_split=3, missing=None, n_estimators=40, n_jobs=-1,
             nthread=None, objective='reg:linear', random_state=1, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

#### Fitting

In [ ]:
grid_xgb.fit(X_train,y_train)

#### Prediction

In [ ]:
grid_xgb_pred = grid_xgb.predict(X_test)

#### Accuracy

In [ ]:
# RMSE and MAE
grid_xgb_rmse = np.sqrt(mean_squared_error(y_test, grid_xgb_pred))
grid_xgb_mae = mean_absolute_error(y_test, grid_xgb_pred)

print("RMSE:", grid_xgb_rmse)
print("MAE:", grid_xgb_mae)

In [ ]:
# Check R2 score
r2_grid_xgb = r2_score(y_test, grid_xgb_pred)
r2_grid_xgb

The parameter turning improved the performance of prediction!!

In [ ]:
# Scatter plot
fig = plt.figure(figsize=(6,5))

plt.scatter(y_test,grid_xgb_pred,color="blue")
x = np.arange(0, 40000)
plt.plot(x,x,color="red")
plt.title("predicted_data vs test_data")
plt.xlabel("test_data")
plt.ylabel("predicted_data")

plt.show()


Generally the predicted values follow the red line.

In [ ]:
# Residual plot
plt.scatter(grid_xgb_pred, grid_xgb_pred - y_test, color = 'blue')     
plt.hlines(y = 0, xmin = -2000, xmax = 38000, color = 'black') 
plt.title('Residual Plot')                                
plt.xlabel('Predicted Values')                            
plt.ylabel('Residuals')                                   
plt.grid()                                                

plt.show()                                               

### LightGBM

In [ ]:
# Default
lgb_model

#### Grid Search

In [ ]:
## #Grid_Search
# params = {
#        'n_estimators'      : [40,45,50],
#        'random_state'      : [1],
#        'n_jobs'            : [-1],
#        'max_depth'         : [2,3,4],
#        'num_leaves'    : [5,10,15]
# }

# grid_lgb = GridSearchCV(estimator=lgb.LGBMRegressor(random_state=1), param_grid=params, \
#                        cv = 10, scoring = 'r2')
# grid_lgb.fit(X_train, y_train)
# print(grid_lgb.best_estimator_)

In [ ]:
# Best estimator
grid_lgb = lgb.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=3,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=45, n_jobs=-1, num_leaves=10, objective=None,
              random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

#### Fitting

In [ ]:
grid_lgb.fit(X_train,y_train)

#### Prediction

In [ ]:
grid_lgb_pred = grid_lgb.predict(X_test)

#### Accuracy

In [ ]:
# RMSE and MAE
grid_lgb_rmse = np.sqrt(mean_squared_error(y_test, grid_lgb_pred))
grid_lgb_mae = mean_absolute_error(y_test, grid_lgb_pred)

print("RMSE:", grid_lgb_rmse)
print("MAE:", grid_lgb_mae)

In [ ]:
# Check R2 score
r2_grid_lgb = r2_score(y_test, grid_lgb_pred)
r2_grid_lgb

R2 score increased by 2% compared with that of the base model.

In [ ]:
# Scatter plot
fig = plt.figure(figsize=(6,5))

plt.scatter(y_test,grid_lgb_pred,color="blue")
x = np.arange(0, 40000)
plt.plot(x,x,color="red")
plt.title("predicted_data vs test_data")
plt.xlabel("test_data")
plt.ylabel("predicted_data")

plt.show()


In [ ]:
# Residual plot
plt.scatter(grid_rf_pred, grid_xgb_pred - y_test, color = 'blue')       
plt.hlines(y = 0, xmin = -2000, xmax = 38000, color = 'black') 
plt.title('Residual Plot')                              
plt.xlabel('Predicted Values')                            
plt.ylabel('Residuals')                                   
plt.grid()                                                
plt.show()                                               

## Interpretation

### Features Importance

In [ ]:
features = X_train.columns
importances = rf_model.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,10))
plt.barh(range(len(indices)), importances[indices],  align='center')
plt.yticks(range(len(indices)), features[indices])
plt.show()

`bmi` `age` `smoker` seem to be quite important.

### Summary

Regression task：
- The most accurate model：LightGBM
- Accuracy：R2 score 88%

### The work left to be done

We can do the below things to improve our work!
1. How are the models improved more?
2. What kind of new experiments will be valuable?

1.
 - Check the outliers in the existing experiment
 - Parameter tuning should be more done?
 - features engineering?
 
2.
 - More variables, more data points
 - Acquire more domain knowledge

## New case

Assume there is a new client applying for the contract.

### Profile

In [ ]:
new_client = pd.DataFrame({"id":10000, "age":58, "sex":1, "bmi":22, "children":2, "smoker":1,
                           "northeast":1, "northwest":0, "southeast":0, "southwest":0}, index=[0])
new_client = new_client.set_index("id")

In [ ]:
new_client

### Prediction with LightGBM

In [ ]:
lgb_new_client = lgb_model.predict(new_client)
lgb_new_client

Well, It is about 22000 USD.